In [1]:
#pip install --user git+https://github.com/LAL/trackml-library

In [1]:
%load_ext autoreload
%autoreload 2

import torch
"""def format_pytorch_version(version):
    return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

#def format_cuda_version(version):
#    return 'cu' + version.replace('.', '')

#CUDA_version = torch.version.cuda
#CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric """

"def format_pytorch_version(version):\n    return version.split('+')[0]\n\nTORCH_version = torch.__version__\nTORCH = format_pytorch_version(TORCH_version)\n\n#def format_cuda_version(version):\n#    return 'cu' + version.replace('.', '')\n\n#CUDA_version = torch.version.cuda\n#CUDA = format_cuda_version(CUDA_version)\n\n!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html\n!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html\n!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html\n!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html\n!pip install torch-geometric "

In [2]:
#!pip install -U imbalanced-learn

In [51]:
#basic libraries
import numpy as np 
import pandas as pd
import trackml.dataset
import sys    
import os
import shutil
import time
#proccesing data 
from data_preprocess import *    # reads data from data_preprocess.py
#ML libraries
# from imblearn.under_sampling import RandomUnderSampler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report
from collections import namedtuple
from torch_geometric.data import Data

from tqdm import tqdm

import warnings
warnings.simplefilter("ignore", UserWarning)


## Data Filters

In [3]:
#######################ENRICH DATA#####################################
# Calculate derived hits variables
def initial_params(df):
    """ Calculate momentum radius, phi and ita"""
    df['pt'] = np.sqrt(df.tpx**2 + df.tpy**2)
    df['r'] = np.sqrt(df.x**2 + df.y**2)
    df['phi'] = np.arctan2(df.y, df.x)
    def calc_ita(r, z):
        theta = np.arctan2(r, z)
        return -1. * np.log(np.tan(theta / 2.)), theta
    df['ita'], df['theta'] = calc_ita(df['r'],df['z'])
    
    return df

def ordered_layer_id(df):
    """order layers from 0 to N"""
    df = df.sort_values(by=['volume_id','layer_id','z']).reset_index(drop = True) 
    new_layers_array = np.zeros(len(df))
    i = 0
    idx = 0
    for vol in df.volume_id.unique():
        temp_df = df.loc[df.volume_id == vol]
        for layer in temp_df.layer_id.unique():
            temp2_df = temp_df.loc[temp_df.layer_id == layer]
            new_layers_array[idx:len(temp2_df)+idx]=i
            i+=1
            idx+=len(temp2_df)
    df['new_layer_id'] = new_layers_array.astype(int)
    return df

In [4]:
#######################SECTION FILTERS#####################################
# firts use the geomerty of the detector to reduce data, barrel, layers, and sectioning
def sections(df,phi_sec,ita_sec, overlap_phi = 0, overlap_ita = 0):
    
    #use only important columns
    df = df[['new_hit_id', 'new_particle_id', 'particle_id','weight','new_layer_id','layer_id','volume_id','x', 'y', 'z','pt','tpx','tpy', 'r', 'phi','ita','event_id']]
    phi_range = (-np.pi, np.pi)
    ita_range =  (-3, 3)
    n_phi_sections = phi_sec          # Towards a realistic track reconstruction algorithm based
    n_ita_sections = ita_sec          #on graph neural networks for the HL-LHC
    
    phi_edges = np.linspace(*phi_range, num=n_phi_sections+1)   #4 sections in range of phi 
    ita_edges = np.linspace(*ita_range, num=n_ita_sections+1)   #2 sections in range of ita
    def split_detector_sections(df, phi_edges, ita_edges):
        """Split hits according to provided phi and ita boundaries."""
        hits_sections = []
        # Loop over sections on phi
        for i in range(len(phi_edges) - 1):
            if np.round(abs(phi_edges[i] - overlap_phi),4)>= 3.1416:     # when angle - overlap is smaller than -pi make angle positive and join dfs in range 
                
                phi_max = phi_edges[i+1] + overlap_phi
                phi_min = ((phi_edges[i] - overlap_phi) + 2*np.pi) 
                pos = df.loc[df.phi > phi_min]
                neg = df.loc[df.phi <= phi_max]
                phi_hits = pd.concat((pos,neg))
                
            elif np.round(abs(phi_edges[i+1] + overlap_phi),4)>= 3.1416:  # when angle + overlap is bigger than pi make angle negative and join dfs in range 
                
                phi_min = phi_edges[i] - overlap_phi
                phi_max = ((phi_edges[i+1] + overlap_phi) - 2*np.pi) 
                pos = df.loc[df.phi > phi_min]
                neg = df.loc[df.phi <= phi_max]
                phi_hits = pd.concat((pos,neg))
                
            else:
                
                phi_min = phi_edges[i] - overlap_phi
                phi_max = phi_edges[i+1] + overlap_phi
                phi_hits = df[(df.phi >= phi_min) & (df.phi < phi_max)]
                
                
            # Center these hits on phi=0
            centered_phi = phi_hits.phi - (phi_min + phi_max) / 2
            phi_hits = phi_hits.assign(sec_phi=centered_phi, phi_section=i)
            #loop over sections on phi
            for j in range(len(ita_edges) - 1):
                ita_min, ita_max = ita_edges[j] - overlap_ita, ita_edges[j+1] + overlap_phi
                # Select hits in this ita section
                sec_hits = phi_hits[(df['ita'] >= ita_min) & (df['ita'] < ita_max)]
                hits_sections.append(sec_hits.assign(ita_section=j))
            #create new hits ids 
        for i ,dfs in enumerate(hits_sections):
            dfs = dfs.reset_index(drop = True)
            dfs.new_hit_id = dfs.index
            hits_sections[i] = dfs
            
        return hits_sections
    return  split_detector_sections(df, phi_edges, ita_edges)


#barrel
def volume_filter(df, volume_list):
    """ 
     Returns only hits in volumes given in volume_list 
    """
    return df.loc[df.volume_id.isin(volume_list)]

#choosen layers
def layer_filter(df, layer_list):
    """ 
     Returns only hits in volumes given in volume_list 
    """
    return df.loc[df.new_layer_id.isin(layer_list)]

In [5]:
#######################INITIAL FILTERS#####################################
def remove_noise(df):
    return df.loc[df.particle_id!=0]
def weight_filter(df):
    """
     Remove weights = 0
    """
    return df.loc[df.weight!=0]

def db_hit_layer_filter(df):
    """
     Remove double hits in the same layer
    """
    return df.loc[df.duplicated(subset=['new_particle_id', 'layer_id','volume_id'])==False]


In [6]:
#######################CALCULATION FILTERS#####################################
def p_t_filter(df, min_pt):
    """Removes particles < min_pt"""
    pt_1 = df.loc[df['pt']>min_pt].new_particle_id.unique()
    return df.loc[df['new_particle_id'].isin(pt_1)]
def ita_filter(df, min_ita, max_ita):
    """keeps particles in the range (min_ita,max_ita)"""
    ita_f = df.loc[(df['ita']<min_ita) | (df['ita']>max_ita)].new_particle_id.unique()
    return df.loc[~df['new_particle_id'].isin(ita_f)]

## Node Edge Conections

In [7]:
#######################MORE THAN 4 FILTERS#####################################
def nhit_filter(df, nhits):
    """
        Remove less than n hits
    """
    #find the amounts of hits per particle
    new_nhits = df.new_particle_id.value_counts().rename_axis('new_particle_id').reset_index(name='new_nhits') 
    #merge the new hhits on new particle id 
    df = df.merge(new_nhits, on='new_particle_id')
    return df.loc[df.new_nhits >= nhits]


In [67]:
#######################CREATE EDGES#####################################
def calc_dphi(phi1, phi2):
    """Computes phi2-phi1 """
    dphi = phi2 - phi1
    return dphi

def get_param_distr(df):
    """ create edges and calculates dphi, dz,dr,dita,dR,phi_slope, and z0"""
    
    dR_list, phi_slope_list,z0_list, hits_pair_list = [], [], [],[]
    #################CREATE EDGES#####################################
    # some particle ids dont have hist in certain layers therefore is not possible to just loop through conecutive layers, 
    # it is neccesary to loop through each p_id to see the set of consecutive layers and conect those layers
    
    for p_id in df.new_particle_id.unique():
        
        temp_df = df.loc[df.new_particle_id==p_id]
        cons_layers = temp_df.new_layer_id.unique()
    # loop trhough layers to generate hits pair(section of track) on consecutive layers 
        for i in range(len(cons_layers)-1):
            
            hits1 = temp_df.loc[temp_df.new_layer_id == cons_layers[i]]
            hits2 = df.loc[df.new_layer_id == cons_layers[i+1]]
            keys = ['event_id', 'new_hit_id','new_layer_id','new_particle_id','r', 'phi', 'z','ita','x','y']
            hit_pairs  = hits1[keys].reset_index(drop = True).merge(hits2[keys].reset_index(drop = True), on='event_id', suffixes=('_1', '_2'))
            
            hit_pairs['y'] = hit_pairs.new_particle_id_1 == hit_pairs.new_particle_id_2
            
            hits_pair_list.append(hit_pairs)
            
        # concatenate firts 2 hit pairs 
    df_concat = pd.concat([hits_pair_list[0], hits_pair_list[1]], ignore_index=True)
    # concatenate firts the rest hit pairs
    for i in range(len(hits_pair_list) -2):
        df_concat = pd.concat([df_concat, hits_pair_list[i+2]], ignore_index=True)
        
    #############CALCULATE PARAMTERS#####################################  
    df_concat['dphi'] = calc_dphi(df_concat.phi_1, df_concat.phi_2)
    df_concat['dz'] = df_concat.z_2 - df_concat.z_1
    df_concat['dr'] = df_concat.r_2 - df_concat.r_1
    df_concat['dita'] = df_concat.ita_2 - df_concat.ita_1
    df_concat['dR'] = np.sqrt(df_concat['dita']**2 + df_concat['dphi']**2)
    df_concat['phi_slope'] = df_concat['dphi'] / df_concat['dr']
    df_concat['z0'] = df_concat.z_1 - df_concat.r_1 * df_concat['dz'] / df_concat['dr']
    return df_concat

In [9]:
def edge_filter(df):
    """reduce amount of edges using calculated parameters"""
    min_z0 =  -320
    max_z0 = 520
    df = df.loc[(df.z0>=min_z0) & (df.z0<=max_z0)]
    max_dr = 0.5
    df = df.loc[df.dR<=max_dr]
    
    min_phi_slope =  -.0055
    max_phi_slope = .0055
    df =df.loc[(df.phi_slope>=min_phi_slope) & (df.phi_slope<max_phi_slope)] 
    return df


## Inbalanced Data Set

In [10]:
#############BALANCE DATASET##################################### 
def resample_data(df):
    """Returns balanced dataset"""
    X = df.drop(['y'], axis=1)
    y = df.y
    
    #undersample data 
    rus = RandomUnderSampler(random_state=42)
    X_res, y_res = rus.fit_resample(X, y)
    
    df = X_res
    df['y'] = y_res
    return df.sort_values(by=['new_layer_id_1','z_1'])

##########RE-LABLE HIT ID#####################################
def New_hit_id(df):
    """returns dataset with ordered hit id"""
    df = df.sort_values(by=['new_layer_id_1','z_1'])
    df_hits = pd.DataFrame()
    df_hits['unique_hits'] =  pd.concat([df['new_hit_id_1'], df['new_hit_id_2']]).unique()
    df_hits = df_hits.sort_values('unique_hits').reset_index(drop = True)
    df_hits['new_hit'] = df_hits.index
    
    newest_id1 = np.zeros(len(df))
    
    for i, id in enumerate(df['new_hit_id_1']):
        newest_id1[i] = df_hits.new_hit.loc[df_hits.unique_hits==id]
    df['hit_id_1'] = newest_id1.astype(int)
    newest_id2 = np.zeros(len(df))
    
    for i, id in enumerate(df['new_hit_id_2']):
        newest_id2[i] = df_hits.new_hit.loc[df_hits.unique_hits==id]
    df['hit_id_2'] = newest_id2.astype(int)
    df = df.sort_values('hit_id_1')
    
    return df

## Data in Graph Format

In [59]:
def data_to_graph(df,directed=True):
    """Returns graph data into pytorch format"""
    edge_attr = df[['dphi', 'dz', 'dr', 'dita', 'dR','phi_slope', 'z0']]
    edge_index = df[['hit_id_1','hit_id_2']]
    y = df.y
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df1[['hit_id','r', 'phi', 'z', 'ita', 'x','y' ]] = df[['hit_id_1','r_1', 'phi_1', 'z_1', 'ita_1','x_1','y_1' ]]
    df2[['hit_id','r', 'phi', 'z', 'ita', 'x','y' ]] = df[['hit_id_2','r_2', 'phi_2', 'z_2', 'ita_2','x_2','y_2' ]]
    x = pd.concat([df1,df2])
    x = x.drop_duplicates(subset=['hit_id'])
    # x = x.sort_values('hit_id')
    x = x[['r', 'phi', 'z', 'ita', 'x','y']]
    
    if directed:
        edge_attr = torch.from_numpy(edge_attr.to_numpy()).type(torch.FloatTensor)
        edge_index = torch.from_numpy(edge_index.to_numpy())   # this is double 
        y = torch.from_numpy(y.to_numpy()).type(torch.FloatTensor)
        x = torch.from_numpy(x.to_numpy()).type(torch.FloatTensor)    #change to float  x[['r', 'phi', 'z', 'ita', 'x','y']]
        data = Data(x=x, edge_index=edge_index.T,edge_attr=edge_attr,y=y)
    else:
        
        edge_index_und = torch.stack([torch.cat([edge_index[:,0], edge_index[:,1]], dim=0), torch.cat([edge_index[:,1], edge_index[:,0]], dim=0)], dim = 0)
        edge_attr_und = torch.cat([edge_attr, -1*edge_attr], dim=0)
        y_und = torch.cat([y,y])
        data = Data(x=x,edge_index= edge_index_und,edge_attr = edge_attr_und,y = y_und)#, first_layer_hits = first_layer_hits  )
    return data 

## Run

In [65]:
from time import time as tt

##########CONSTANTS#####################################
volume_list = [8,13,17]   #barrel
pt_cut = 1
ita_min = -3
ita_max = 3
nhits = 5
layer_list = [0,1,2,3,4,5,7,8]

def run_and_save(event_start,event_end,input_dir, output_dir, path_file):
    """transforms raw data into graph format as outlined in the report and save it"""
    for file_idx, i in enumerate(tqdm(range(event_start,event_end))):
        #read data 
        tic = tt()
        hits, particles, truth = read_all_data(input_dir, 1000 + i,1000 + (i+1))
        df = merge_dfs(hits, particles, truth)

        #firts volume filter
        df = volume_filter(df, volume_list)

        #fits stage create new_layer_id 
        df = ordered_layer_id(df)

        #then return only layers in list 
        df = layer_filter(df, layer_list)

        #calculate the extra parameters
        df = initial_params(df)

        #section detector
        df_sections = sections(df,4,2, overlap_phi = 0.26, overlap_ita = 0.5)

        #loop throug all sections 
        for sec,dfs in enumerate(df_sections):
            #initial filters
            dfs = dfs.pipe(remove_noise).pipe(weight_filter).pipe(db_hit_layer_filter)

            #calculation filter 
            dfs = dfs.pipe(initial_params).pipe(p_t_filter,pt_cut).pipe(ita_filter,ita_min, ita_max)

            #less than 4 hit filter
            dfs = nhit_filter(dfs, nhits)

            #order layers again
            dfs = dfs.pipe(ordered_layer_id)

            # construct graph segments (edges) in the correct range
            dfs = dfs.pipe(get_param_distr)

            dfs = dfs.pipe(edge_filter)

            dfs = dfs.pipe(New_hit_id)

            #create graph data
            data = data_to_graph(dfs)

            #data to graph 
            newpath = os.path.join(output_dir, path_file, f"Section_{sec}")
            if not os.path.exists(newpath):
                os.makedirs(newpath)
            #save grpah files 
            torch.save(data, os.path.join(newpath, f"graph_{file_idx}.pt"))
        

In [68]:
#train
input_dir = "/global/cfs/cdirs/m3443/data/trackml-kaggle/train_100_events/"
output_dir = "/global/cfs/cdirs/m3443/data/EventFilterGNN/InitialExplore"
run_and_save(0,80, input_dir, output_dir, 'Train')
run_and_save(80,90, input_dir, output_dir, 'Test')
run_and_save(90,100, input_dir, output_dir, 'Val')

  0%|          | 0/80 [00:00<?, ?it/s]

read data 0.14872097969055176
volume filter 0.004141569137573242
ordered layer id 0.020791053771972656
layer filter 0.0020678043365478516
initial params 0.005346536636352539
nhit filter 0.03371024131774902
ordered layer id 0.003699064254760742
get param distr 2.0757715702056885
edge filter 0.008132219314575195
new hit id 2.1019303798675537
data to graph 0.008325815200805664
save graph 0.004071474075317383
nhit filter 0.008943319320678711
ordered layer id 0.003823518753051758
get param distr 2.379948139190674
edge filter 0.00879216194152832
new hit id 2.544893264770508
data to graph 0.008504390716552734
save graph 0.0027244091033935547
nhit filter 0.009164571762084961
ordered layer id 0.003598451614379883
get param distr 1.908984661102295
edge filter 0.0049571990966796875
new hit id 2.093327045440674
data to graph 0.008273839950561523
save graph 0.0038132667541503906
nhit filter 0.009382486343383789
ordered layer id 0.0038118362426757812
get param distr 2.190713882446289
edge filter 0.0

  1%|▏         | 1/80 [00:38<51:04, 38.80s/it]

new hit id 3.019045352935791
data to graph 0.008684396743774414
save graph 0.0035719871520996094
read data 0.131087064743042
volume filter 0.004152536392211914
ordered layer id 0.017221689224243164
layer filter 0.0027735233306884766
initial params 0.004601478576660156
nhit filter 0.02969193458557129
ordered layer id 0.0033745765686035156
get param distr 1.2769315242767334
edge filter 0.0038161277770996094
new hit id 0.927178144454956
data to graph 0.007242918014526367
save graph 0.002352476119995117
nhit filter 0.008741617202758789
ordered layer id 0.0035839080810546875
get param distr 1.3084239959716797
edge filter 0.00354766845703125
new hit id 0.7515738010406494
data to graph 0.0069732666015625
save graph 0.002716064453125
nhit filter 0.008315563201904297
ordered layer id 0.0035665035247802734
get param distr 1.2824594974517822
edge filter 0.003992319107055664
new hit id 1.0276944637298584
data to graph 0.007283210754394531
save graph 0.0024390220642089844
nhit filter 0.008416652679

  2%|▎         | 2/80 [00:58<35:30, 27.32s/it]

...

new hit id 1.9033448696136475
data to graph 0.007612943649291992
save graph 0.0012998580932617188
read data 0.23740434646606445
volume filter 0.003337383270263672
ordered layer id 0.018718242645263672
layer filter 0.00171661376953125
initial params 0.005125999450683594
nhit filter 0.029535770416259766
ordered layer id 0.0039784908294677734
get param distr 1.9539027214050293
edge filter 0.005366802215576172
new hit id 1.9674530029296875
data to graph 0.008040666580200195
save graph 0.0012750625610351562
nhit filter 0.00942373275756836
ordered layer id 0.0043871402740478516
get param distr 2.475355625152588
edge filter 0.008398771286010742
new hit id 2.9730896949768066
data to graph 0.008621692657470703
save graph 0.0014262199401855469
nhit filter 0.008710861206054688
ordered layer id 0.0041010379791259766
get param distr 1.6596167087554932
edge filter 0.0044193267822265625
new hit id 1.8629026412963867
data to graph 0.008393049240112305
save graph 0.0013320446014404297
nhit filter 0.009

 90%|█████████ | 9/10 [05:30<00:36, 36.58s/it]

new hit id 1.1006288528442383
data to graph 0.007965326309204102
save graph 0.0016782283782958984
read data 0.3074305057525635
volume filter 0.0048007965087890625
ordered layer id 0.021353483200073242
layer filter 0.002027750015258789
initial params 0.005595207214355469
nhit filter 0.03155851364135742
ordered layer id 0.004120349884033203
get param distr 2.2165470123291016
edge filter 0.007735729217529297
new hit id 2.5829062461853027
data to graph 0.008644819259643555
save graph 0.0012624263763427734
nhit filter 0.009346961975097656
ordered layer id 0.00395965576171875
get param distr 2.7082457542419434
edge filter 0.011412620544433594
new hit id 2.910914421081543
data to graph 0.008260726928710938
save graph 0.0013282299041748047
nhit filter 0.008911848068237305
ordered layer id 0.004013776779174805
get param distr 2.2278215885162354
edge filter 0.005652666091918945
new hit id 2.595388889312744
data to graph 0.007932186126708984
save graph 0.0014235973358154297
nhit filter 0.00863838

100%|██████████| 10/10 [06:12<00:00, 37.27s/it]

new hit id 2.0162851810455322
data to graph 0.007856369018554688
save graph 0.0013077259063720703
